**Importing Libraries**

In [164]:
import pandas as pd
import numpy as np
import random

# Definindo a semente para reprodução
np.random.seed(42)
random.seed(42)

**Reading Files**

In [165]:
magic_items = pd.read_csv('data/magic_items.csv', sep=';')
adventure_gear = pd.read_csv('data/adventuring_gear.csv', sep=';')
armor = pd.read_csv('data/armor.csv', sep=';')
poisons = pd.read_csv('data/poisons.csv', sep=';')
potions = pd.read_csv('data/potions.csv', sep=';')
weapons = pd.read_csv('data/weapons.csv', sep=';')
names = pd.read_csv('data/names.csv', sep=';')

In [166]:
magic_items.head()

,item_id,Name,Price,Rarity,Category
0,001-ACo,Ammunition +1 (Per),15 gp,Uncommon,Consumable Items
1,002-ACo,Ammunition +2 (Per),50 gp,Rare,Consumable Items
2,003-ACo,Ammunition +3 (Per),250 gp,Very Rare,Consumable Items
3,004-ACo,Arrow of Slaying,400 gp,Very Rare,Consumable Items
4,005-BCo,Bead of Force,"1,000 gp",Rare,Consumable Items


In [167]:
adventure_gear.head()

,item_id,Name,Price,Weight,type
0,01-Ars,Abacus,2 gp,2 lb.,Others
1,02-Ars,Acid (vial),25 gp,1 lb.,Others
2,03-Ars,Alchemist's Fire (flask),50 gp,1 lb.,Others
3,04-Aon,Arrows (20),1 gp,1 lb.,Ammunition
4,05-Bon,Blowgun Needle (50),1 gp,1 lb.,Ammunition


In [168]:
weapons.head()

,item_id,Name,Price,Damage,Weight,Properties,type
0,01-Cns,Club,1 sp,1d4 Bludgeon,2 lb.,Light,Simple Melee Weapons
1,02-Dns,Dagger,2 gp,1d4 Piercing,1 lb.,"Finesse, Light, Thrown (20/60)",Simple Melee Weapons
2,03-Gns,Greatclub,2 sp,1d8 Bludgeon,10 lb.,Two-handed,Simple Melee Weapons
3,04-Hns,Handaxe,5 gp,1d6 Slashing,2 lb.,"Light, Thrown (20/60)",Simple Melee Weapons
4,05-Jns,Javelin,5 sp,1d6 Piercing,2 lb.,Thrown (30/120),Simple Melee Weapons


In [169]:
armor.head()

,item_id,Name,Price,AC,Weight,Requirements,Stealth,type
0,01-Por,Padded,5 gp,11 + Dex,8 lb.,NaN,Disadvantage,Light Armor
1,02-Lor,Leather,10 gp,11 + Dex,10 lb.,NaN,NaN,Light Armor
2,03-Sor,Studded Leather,45 gp,12 + Dex,13 lb.,NaN,NaN,Light Armor
3,04-Hor,Hide,10 gp,12 + Dex(max2),12 lb.,NaN,NaN,Medium Armor
4,05-Cor,Chain Shirt,50 gp,13 + Dex(max2),20 lb.,NaN,NaN,Medium Armor


In [170]:
potions.head()

,item_id,Name,Price,Rarity
0,01-Pon,Potion of Healing,50 gp,Common
1,02-Pon,Potion of Greater Healing,150 gp,Uncommon
2,03-Pre,Potion of Superior Healing,450 gp,Rare
3,04-Pre,Potion of Supreme Healing,"1,350 gp",Very Rare
4,05-Ere,Elixir of Health,120 gp,Rare


In [171]:
poisons.head()

,item_id,Name,Price,Type,DC
0,01-Aed,Assassin's blood,150 gp,Ingested,10.0
1,02-Ted,Truth serum,150 gp,Ingested,11.0
2,03-Cct,Carrion crawler mucus,200 gp,Contact,13.0
3,04-Dry,Drow poison,200 gp,Injury,13.0
4,05-Sry,Serpent venom,200 gp,Injury,11.0


<hr style="height:3px;border-width:0;color:blue;background-color:blue">

# **Data Creation**

**Passos:**

1. Filtrar as Raridades dos itens
1. Criar uma tabela que contenha todos os produtos
    * A tabela deve ter apenas informações básicas sobre os itens, como: id | nome | preço | tipo.
1. Criar uma tabela com as informações dos clientes
1. Criar a tabela fato de **vendas**




### **Filtrar Raridades:**

In [172]:
magic_items['Rarity'].value_counts()

Rare         114
Uncommon      83
Very Rare     63
Legendary     38
Common         2
Name: Rarity, dtype: int64

Como eu pretendo simular as vendas de um vendedor comum, não vou usar itens que sejam: Very Rare, Legendary.

In [173]:
# função para filtrar a raridade dos items
def filter_rarity(dataframe):
    unwanted = ['Very Rare', 'Legendary']
    df = dataframe.copy()
    df = df.query("Rarity not in @unwanted")
    return df

In [174]:
magic_items_filtered = filter_rarity(magic_items)

In [175]:
magic_items_filtered['Rarity'].value_counts()

Rare        114
Uncommon     83
Common        2
Name: Rarity, dtype: int64

In [176]:
potions_filtered =  filter_rarity(potions)

In [177]:
potions_filtered['Rarity'].value_counts()

Uncommon    10
Rare        10
Common       2
Name: Rarity, dtype: int64

Nenhum outro Dataset tem a coluna 'Rarity'

### **Criar tabela Produtos**

Primeiramente devo mudar as colunas 'type' para 'cetegory', depois criar uma nova coluna 'type' com o tipo de item de cada dataset.

In [178]:
# função para substituir a coluna type e colocar os nomes das colunas em minúsculo
def replace_lower_column(df):
    df = df.columns.str.lower().str.replace('type', 'category')
    return df

In [179]:
adventure_gear.columns = replace_lower_column(adventure_gear)
magic_items_filtered.columns = replace_lower_column(magic_items_filtered)
armor.columns = replace_lower_column(armor)
weapons.columns = replace_lower_column(weapons)
potions_filtered.columns =  replace_lower_column(potions_filtered)
poisons.columns = replace_lower_column(poisons)

**Criando as novas colunas 'type':**

In [180]:
adventure_gear['type'] = 'adventure_gear'
magic_items_filtered['type'] = 'magic_item'
weapons['type'] = 'weapon'
potions_filtered['type'] = 'potion'
poisons['type'] = 'poison'
armor['type'] = 'armor'

**Criando a tabela Produtos:**

| id | nome | preço | tipo |


In [181]:
wanted_cols = ['item_id', 'name', 'price', 'type']

In [182]:
product = magic_items_filtered[wanted_cols].copy()

In [183]:
product = pd.concat([product,
                     adventure_gear[wanted_cols],
                     weapons[wanted_cols],
                     armor[wanted_cols],
                     potions_filtered[wanted_cols],
                     poisons[wanted_cols]],
                     ignore_index=True)

In [184]:
product['type'].value_counts()

magic_item        199
adventure_gear    108
weapon             37
potion             22
poison             16
armor              13
Name: type, dtype: int64

In [185]:
product.head()

,item_id,name,price,type
0,001-ACo,Ammunition +1 (Per),15 gp,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,magic_item
2,005-BCo,Bead of Force,"1,000 gp",magic_item
3,006-CCo,Chime of Opening,400 gp,magic_item
4,007-DCo,Deck of Illusions,900 gp,magic_item


In [186]:
# testando o relacionamento das tabelas
MI_itemID =  random.choices(product['item_id'].loc[product['type'] == 'magic_item'])
magic_items_filtered.query('item_id == @MI_itemID')

,item_id,name,price,rarity,category,type
192,193-ICo,Ioun Stone Awareness,500 gp,Rare,Combat Items,magic_item


O relacionamento está funcionando como o desejado.

## **Criar a Tabela Cliente**

**Atributos do cliente:**


* id
* nome
* sexo
* idade
* raça
* classe
* endereço (cidade) (talvez)
* contato (talvez)

In [187]:
# checando duplicatas
names.duplicated().sum()

140

In [188]:
names.drop_duplicates(inplace=True)

In [189]:
names.shape

(1423, 2)

referência = https://bg3.wiki/wiki/Races

In [190]:
races = pd.read_csv('data/races.csv', sep=';')
races

,race,base_age,max_age,maximum_age_range
0,Dragonborn,15,80,80 + 1d20
1,Drow,80,225,225 + 3d100
2,Dwarf,50,350,300 + 2d100
3,Elf,90,750,425 + 5d100
4,Githyanki,30,250,250 + 1d100
5,Gnome,60,200,200 + 3d100
6,Half-Elf,15,125,125 + 3d20
7,Halfling,20,100,100 + 1d100
8,Half-Orc,12,60,90 + 2d20
9,Human,15,90,90 + 2d20


referência: https://www.dndbeyond.com/sources/basic-rules/classes#ClassesSummary

In [191]:
classes = pd.read_csv('data\classes.csv', sep=';')
classes

,Class,Armor,Weapon
0,Barbarian,"Light armor, medium armor, shields","Simple weapons, martial weapons"
1,Bard,Light armor,"simple weapons, hand crossbows, longswords, ra..."
2,Cleric,"Light armor, medium armor, shields",Simple weapons
3,Druid,"Light armor, medium armor, shields","Clubs, daggers, darts, javelins, maces, quarte..."
4,Fighter,"Light armor, medium armor, heavy armor, shields","Simple weapons, martial weapons"
5,Monk,None,"Simple weapons, shortswords"
6,Paladin,"Light armor, medium armor, heavy armor, shields","Simple weapons, martial weapons"
7,Ranger,"Light armor, medium armor, shields","Simple weapons, martial weapons"
8,Rogue,Light armor,"Simple weapons, hand crossbows, longswords, ra..."
9,Sorcerer,None,"Daggers, darts, slings, quarterstaffs, light c..."


A tabela de cliente deve ter a seguinte estrutura:

| customer_id| name    | sex      | race     | age     | class   |
| -------- | ------- | -------- | ------- | -------- | ------- |
| -------- |-------- | -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |

**Função para gerar IDs:**

In [192]:
import time
import string

# criando função para gerar IDs
def generate_custom_id(i):
    """Function to generate random IDs"""
    timestamp = str(int(time.time()) + i)
    random_chars = ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))
    return f"{timestamp[4:]}-{random_chars}"   # return new unique ID

In [193]:
for i in range(20):
    print(generate_custom_id(i))

327454-AJI0Y6
327455-DPBHSA
327456-HXTHV3
327457-A3ZMF8
327458-MDD4V3
327459-0T9NT3
327460-W5UZBI
327461-KCIDKW
327462-NNHJ7X
327463-VG0FN9
327464-XUY41I
327465-BLJH75
327466-LXO6QJ
327467-IUJV6O
327468-H9SDBD
327469-W2PCN9
327470-T84AZY
327471-TJXEPQ
327472-85JSG6
327473-5KXVF1


In [194]:
# random ID
custom_id = generate_custom_id(1)
print(custom_id)

327455-T2TALA


In [195]:
# criando um dataframe com os randoms IDs
customers = pd.DataFrame({
    'customer_id': [generate_custom_id(i) for i in range(1, names.shape[0] + 1)]})
customers

,customer_id
0,327455-753LC5
1,327456-8DRC11
2,327457-ERTJ5P
3,327458-HT0HL9
4,327459-XPSEIM
...,...
1418,328873-CTM09H
1419,328874-ZBGTJ5
1420,328875-4696QZ
1421,328876-6CP7SA


In [196]:
# checando IDs duplicados
customers.duplicated().sum()

0

In [197]:
# verificando NAs
names.isna().sum()

name    0
sex     0
dtype: int64

In [198]:
# removendo espaços antes e depois
names['name'] = names['name'].str.strip()

In [199]:
# resetando os índices
names.reset_index(drop=True, inplace=True)

In [200]:
# inserindo os nomes e sexos na tabela de clientes
customers["name"] = names['name']
customers["sex"] = names['sex']

In [201]:
customers

,customer_id,name,sex
0,327455-753LC5,Veklani Daargen,female
1,327456-8DRC11,Kasaki Wygarthe,female
2,327457-ERTJ5P,Rosalyn Faringray,female
3,327458-HT0HL9,Atalya Webb,female
4,327459-XPSEIM,Grenenzel Lyfalia,female
...,...,...,...
1418,328873-CTM09H,Arnan Ramcrown,male
1419,328874-ZBGTJ5,Tavon Stormchapel,male
1420,328875-4696QZ,Dodd Fryft,male
1421,328876-6CP7SA,Markus Shattermast,male


In [202]:
races['race'].unique()

array(['Dragonborn\xa0', 'Drow', 'Dwarf', 'Elf', 'Githyanki', 'Gnome',
       'Half-Elf', 'Halfling', 'Half-Orc', 'Human', 'Tiefling'],
      dtype=object)

In [203]:
# limpando a coluna 'race'
races['race'] = races['race'].str.replace('\xa0', '').str.strip()

In [204]:
customers['race'] = [random.choice(list(races['race'])) for _ in range(names.shape[0])]

Agora vem uma etapa não tão simples, a idade, cada raça tem um período de vida diferente e pretendo conservar essas características também.

In [205]:
# criando a função que irá gerar a idade aleatória baseada da raça que foi dada
def random_age(X):
    """This function returns a random age based on the race"""
    race_infos = races[races['race'] == X].reset_index(drop=True)
    return random.randint(race_infos['base_age'][0], race_infos['max_age'][0])

In [206]:
customers['age'] = customers['race'].apply(random_age)

Agora resta apenas escolher as classes para cada cliente.

In [207]:
customers['class'] = [random.choice(list(classes['Class'])) for _ in range(customers.shape[0])]

Conferindo a estrutura da tabela:

| customer_id| name    | sex      | race     | age     | class   |
| -------- | ------- | -------- | ------- | -------- | ------- |
| -------- |-------- | -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |
| -------- | --------| -------- | ------- | -------- | ------- |

In [208]:
customers.sample(n=10, random_state=42)

,customer_id,name,sex,race,age,class
1185,328640-ZAQ7QH,Vzani Van Hyden,female,Dwarf,337,Ranger
677,328132-5R96KH,Marnia Skandalor,female,Tiefling,37,Paladin
1084,328539-Q8VA8X,Lokara Talandro,female,Gnome,118,Sorcerer
1005,328460-40KZUJ,Sevenson Zereni,male,Elf,705,Monk
944,328399-MRSEZY,Talfen Ronefel,male,Half-Orc,23,Rogue
538,327993-7OI0E0,Vauldra Velene,female,Half-Orc,54,Wizard
994,328449-UDUAHG,Dalkon Romazi,male,Gnome,168,Cleric
724,328179-212PGY,Mirabel Yellowcrane,female,Half-Orc,26,Bard
70,327525-OZCYWD,Lokara Strong,female,Dragonborn,25,Barbarian
123,327578-2KNFTU,Meklan Van Devries,male,Elf,146,Cleric


## **Criar a tabela de Vendas**

In [209]:
from datetime import datetime, timedelta

**Minhas tabelas:**
* adventure_gear
* magic_items_filtered
* weapons
* potions_filtered
* poisons
* armor
* product
* costumers

A tabela sales deve seguir a estrutura:

| sale_id  | date    | customer_id | product_id    | product_name | quantity| product_price | 
| -------- | ------- | ----------- | --------------| -------------| ------- |-------------- |
| -------- |-------- | ----------- | --------------| -------------| ------- |-------------- |
| -------- | --------| ----------- | --------------| -------------| ------- |-------------- |
| -------- | --------| ----------- | --------------| -------------| ------- |-------------- |

**Função para gerar IDs:**

In [210]:
# criando função para gerar IDs para tabela sales
def generate_sale_id(i):
    """Function to generate random sale_IDs.
    
    This will return a random string with length = 11 chars"""
    timestamp = str(int(time.time_ns()) + i)
    random_chars = ''.join(random.choices(string.ascii_uppercase + string.digits, k=5))
    return f"{timestamp[13:]}-{random_chars}"   # return new unique ID

In [211]:
print("Exemplo de sale_id: ", generate_sale_id(1))

Exemplo de sale_id:  972501-2BHON


In [212]:
product['price'].unique()

array(['15 gp', '50 gp', '1,000 gp', '400 gp', '900 gp', '300 gp',
       '100 gp', '600 gp', '350 gp', '450 gp', '150 gp', '500 gp',
       '1,600 gp', '4,000 gp', '8,000 gp', '120 gp', '200 gp', '25 gp',
       '35 gp', '280 gp', '570 gp', '2,640 gp', '5,280 gp', '275 gp',
       '2,000 gp', '1,250 gp', '1,300 gp', '800 gp', '5,000 gp',
       '1,500 gp', '250 gp', '750 gp', '700 gp', '1,350 gp', '3,000 gp',
       '4,500 gp', '6,000 gp', '5,600 gp', '1,200 gp', '1,0500 gp',
       '550 gp', '3,500 gp', '2,0500 gp', 'Armor + 500gp',
       'Armor + 500 gp', 'Armor + 800 gp', 'Armor + 200 gp', '2,500 gp',
       '40,000 gp', '2 gp', '1 gp', '4 cp', '10 gp', '20 gp', '5 gp',
       '4 sp', '5 sp', '5 cp', '1 cp', '2 cp', ' 5 gp', '1 sp', '2 sp',
       '4 gp', '1000 gp', '30 gp', '6 gp', '3 gp', '12 gp', '75 gp',
       '45 gp', '1500 gp', '450 gp ', '90 gp', '180 gp', '270 gp',
       '960 gp', '1,920 gp', '480 gp'], dtype=object)

Alguns preços possuem informações que estão despadrozinadas como "Armor + 200 gp", precisa ser limpados da tabela de produtos. 

In [213]:
# removendo observações no campo de preço
product['price'] = product['price'].str.replace('Armor + ', '', regex=False).str.strip()

In [214]:
product['price'].unique()

array(['15 gp', '50 gp', '1,000 gp', '400 gp', '900 gp', '300 gp',
       '100 gp', '600 gp', '350 gp', '450 gp', '150 gp', '500 gp',
       '1,600 gp', '4,000 gp', '8,000 gp', '120 gp', '200 gp', '25 gp',
       '35 gp', '280 gp', '570 gp', '2,640 gp', '5,280 gp', '275 gp',
       '2,000 gp', '1,250 gp', '1,300 gp', '800 gp', '5,000 gp',
       '1,500 gp', '250 gp', '750 gp', '700 gp', '1,350 gp', '3,000 gp',
       '4,500 gp', '6,000 gp', '5,600 gp', '1,200 gp', '1,0500 gp',
       '550 gp', '3,500 gp', '2,0500 gp', '500gp', '2,500 gp',
       '40,000 gp', '2 gp', '1 gp', '4 cp', '10 gp', '20 gp', '5 gp',
       '4 sp', '5 sp', '5 cp', '1 cp', '2 cp', '1 sp', '2 sp', '4 gp',
       '1000 gp', '30 gp', '6 gp', '3 gp', '12 gp', '75 gp', '45 gp',
       '1500 gp', '90 gp', '180 gp', '270 gp', '960 gp', '1,920 gp',
       '480 gp'], dtype=object)

In [215]:
product

,item_id,name,price,type
0,001-ACo,Ammunition +1 (Per),15 gp,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,magic_item
2,005-BCo,Bead of Force,"1,000 gp",magic_item
3,006-CCo,Chime of Opening,400 gp,magic_item
4,007-DCo,Deck of Illusions,900 gp,magic_item
...,...,...,...,...
390,12-Wry,Wyvern poison,"1,200 gp",poison
391,13-Med,Midnight tears,"1,500 gp",poison
392,14-Pry,Purple worm poison,"2,000 gp",poison
393,15-Aed,Antitoxin,50 gp,poison


In [216]:
# Criando a tabela de datas para simular vendas ao longo do tempo
data_inicial = datetime(2017, 1, 1)
data_final = datetime(2023, 12, 31)
intervalo_datas = [data_inicial + timedelta(days=i) for i in range((data_final - data_inicial).days + 1)]

A data inicial foi inspirada na data inicial do desenvolvimento do jogo Baldur's Gate 3.

referência: https://larian.com/support/faqs/general-information_46

In [217]:
sales = pd.DataFrame({
          'sale_id': [generate_sale_id(i) for i in range(50_832)],
          'date': [random.choice(intervalo_datas) for _ in range(50_832)],
          'customer_id': [random.choice(list(customers['customer_id'])) for _ in range(50_832)],
          'product_id': [random.choice(list(product['item_id'])) for _ in range(50_832)],
          'quantity': [random.randint(1, 5) for _ in range(50_832)]
          })

A quantidade de items comprados é baseado na quantidade base de itens inicias dos personagens nas campanhas  de Dungeons and Dragons.

A quantidade de vendas foi baseada no número de avaliações do ***Player's Handbook: Everything a Player Needs to Create Heroic Characters for the World's Greatest Roleplaying Game***.

Referência: https://www.amazon.com.br/Players-Handbook-Wizards-RPG-Team/dp/0786965606/ref=sr_1_7?keywords=dnd+5e&qid=1701899431&refinements=p_n_feature_nine_browse-bin%3A8529758011&rnid=8529757011&s=books&sr=1-7&ufe=app_do%3Aamzn1.fos.db68964d-7c0e-4bb2-a95c-e5cb9e32eb12

In [218]:
sales

,sale_id,date,customer_id,product_id,quantity
0,019600-4WBAB,2019-01-03,328842-Z0I083,062-BNo,1
1,019601-Q0FOT,2020-10-10,328144-E58HUX,09-Sns,2
2,019602-10GDM,2018-03-21,328712-ZZKW3G,86-Srs,2
3,019603-IPC8Q,2018-11-15,327646-NKOMV2,19-Brs,4
4,019604-O6P43,2023-10-19,327979-DEJ31G,296-ICo,1
...,...,...,...,...,...
50827,686727-R2NR0,2017-08-24,328851-FERFXI,09-Pon,1
50828,686728-HV4GY,2021-11-09,327894-IRC9ZD,058-BNo,1
50829,686729-PBLV1,2022-02-02,328252-SN2S8N,286-PCo,4
50830,686730-QET6W,2020-11-22,328133-UB2RTS,107-RNo,3


In [219]:
# renomeando a coluna na tabela produto para dar match na tabela de vendas
product.rename(columns={'item_id': 'product_id',
                        'name': 'product_name'}, inplace=True)

In [220]:
# adiconando os preços e nomes dos produtos na tabela vendas
sales = pd.merge(sales, product[['product_id', 'price', 'product_name']], on='product_id', how='left')

In [221]:
# verificando uma amostra aleatória das vendas
sales.sample(n=10, random_state=42)

,sale_id,date,customer_id,product_id,quantity,price,product_name
18577,887677-KBSB0,2023-08-11,328824-PZTC09,177-WCo,3,400 gp,Weapon of Warning
37264,814564-D4G1T,2021-04-19,328305-WAFFTO,026-NCo,3,"1,600 gp",Necklace of Fireballs (4 beads)
46966,714766-BNGP1,2019-05-17,328390-Z40AQ1,084-ENo,1,250 gp,Eyes of the Eagle
38305,771505-ANC17,2023-02-09,328452-GMNHIE,85-Srs,5,5 gp,"Scale, merchant's"
24865,851665-F2NAN,2022-02-23,328510-2EOWLC,105-QNo,1,250 gp,Quiver of Ehlonna
15132,914732-KUTCS,2019-10-27,327643-Q3XLRF,05-Jns,4,5 sp,Javelin
43040,740540-7MIGD,2021-02-18,328257-IVROCV,99-Wrs,1,1 cp,Whetstone
14164,921464-JFJXE,2020-04-28,327457-ERTJ5P,254-SCo,5,150 gp,Saddle of the Cavalier
33394,797094-TTXDO,2023-01-05,328623-LOLN3E,132-BSu,2,"4,000 gp",Bronze Griffon
49037,703337-V9JK6,2023-10-04,327550-7PTX63,023-NCo,4,300 gp,Necklace of Fireballs (1 bead)


Conferindo a estrutura da tabela:

| sale_id  | date    | customer_id | product_id    | product_name | quantity| product_price | 
| -------- | ------- | ----------- | --------------| -------------| ------- |-------------- |
| -------- |-------- | ----------- | --------------| -------------| ------- |-------------- |
| -------- | --------| ----------- | --------------| -------------| ------- |-------------- |

## **Gerando influência nos dados:**

Essa etapa visa gerar vendas baseada em ocorrências externas, vou utilizar um dataset que eu mesmo criei visando influênciar nas vendas da tabela original.

O Dataset contém relatos de encontros aleatórios nas proximidades da cidade, com informações sobre datas, tipos de monstros e tipos de encontros.

importando a tabela:

In [222]:
encounters = pd.read_csv('data/encounters.csv', parse_dates=['date'])

In [223]:
encounters.head()

,enc_id,date,monster,encounter_type
0,7900-C4E5,2021-09-10,owlbear,combat
1,7901-HRDU,2019-08-19,owlbear,combat
2,7902-4XSZ,2022-06-21,owlbear,sighting
3,7903-JNVC,2023-07-04,owlbear,sighting
4,7904-7HR6,2023-08-14,owlbear,sighting


In [224]:
# quantidade de encontros
count_encounters =  len(encounters)

In [225]:
encounters_sales = pd.DataFrame({
          'sale_id': [generate_sale_id(i) for i in range(count_encounters)],
          'date': [random.choice(list(encounters['date'])) for _ in range(count_encounters)],
          'customer_id': [random.choice(list(customers['customer_id'])) for _ in range(count_encounters)],
          'product_id': [random.choice(list(product['product_id'])) for _ in range(count_encounters)],
          'quantity': [random.randint(1, 5) for _ in range(count_encounters)]
          })

In [226]:
# adiconando os preços e nomes dos produtos na tabela
encounters_sales = pd.merge(encounters_sales, product[['product_id', 'price', 'product_name']], on='product_id', how='left')

Agora vou alterar as datas para um valor aletório de 1 até 7 dias depois das datas originais.

In [227]:
# função para retornar um data aleatória de 1 até 7 dias após a data original
def random_date(x):
    date = x + timedelta(random.randint(1, 7))
    return date

In [228]:
# aplicando a função
encounters_sales['date'] = encounters_sales['date'].apply(random_date)

In [229]:
encounters_sales.sort_values(by='date')

,sale_id,date,customer_id,product_id,quantity,price,product_name
409,288509-YBWKX,2017-05-07,327903-J9BOQ0,101-NNo,5,800 gp,Necklace of Adaption
388,288488-RVJUE,2017-05-11,327461-64CIYH,106-Hnt,5,3 gp,Horn
55,261155-5HD0L,2017-05-23,327651-0YSXPN,127-WNo,2,"4,000 gp",Winged Boots
442,288542-QES4N,2017-05-25,327967-4XYJOA,138-ISu,4,"3,000 gp",Ivory Goat (Terror)
506,288606-BNEV7,2017-06-01,328032-8QEOZ0,78-Qrs,5,1 gp,Quiver
...,...,...,...,...,...,...,...
120,261220-5C575,2023-10-19,328342-Q7W7JP,02-Ted,2,150 gp,Truth serum
21,261121-6BPOC,2023-10-21,328751-VCEQFG,081-ENo,3,200 gp,Eversmoking Bottle
534,288634-1Y7D8,2023-10-24,328640-ZAQ7QH,54-Hrs,3,5 gp,Hunting Trap
0,261100-86BBQ,2023-10-25,328635-I9MQVA,218-TCo,1,"2,0500 gp",Tentacle Rod


Agora basta concatenar as novas vendas geradas na tabela original de vendas.

In [230]:
sales = pd.concat([sales, encounters_sales]).reset_index(drop=True)

A tabela está finalizada, porém antes de transformar em um DB, vou criar algumas inconsistências.

## **Criando incosistências na tabela de vendas:**

**Motivação:**

Resolvi criar algumas incosistências para aumentar o nível de dificuldade para as pessoas que vão usar essa database para praticar análise de dados.

**O que será feito?**
- Gerar duplicatas
- Gerar valores vazios
- Gerar Outliers


### **Gerando Duplicatas:**

In [231]:
# duplicando 7% dos dados de forma aleatória
sales_copy = sales.sample(frac=0.07, random_state=42).copy()

In [232]:
sales_copy.head()

,sale_id,date,customer_id,product_id,quantity,price,product_name
45482,723982-FNK5C,2019-11-24,327777-6T1NTM,100-Bnt,3,30 gp,Bagpipe
40430,755930-WMTOP,2022-12-20,327573-OXFQ8M,17-Brs,2,1 gp,Bedroll
5610,985610-VB0BS,2023-10-30,328134-OYG9ZQ,31-Wns,3,15 gp,Warhammer
3085,998885-PD7CD,2017-06-09,327488-0L7253,10-Bed,3,500 gp,Burnt othur fumes
46761,714561-YE9FR,2020-11-02,327844-9QM3P9,08-Eed,3,300 gp,Essence of ether


Verificando duplicatas na tabela original:

In [233]:
sales.duplicated().sum()

0

In [234]:
sales = pd.concat([sales, sales_copy])

In [235]:
print("Duplicatas na tabela 'sales': ", sales.duplicated().sum())

Duplicatas na tabela 'sales':  3600


### **Gerando valores vazios:**

In [236]:
# verificando se há valores vazios
sales.isna().sum()

sale_id         0
date            0
customer_id     0
product_id      0
quantity        0
price           0
product_name    0
dtype: int64

In [237]:
# escolhendo aleatóriamente os índices das colunas que vou usar
sales_id_NA_idx = sales['sale_id'].sample(frac=0.001, random_state=42).index
customer_id_NA_idx = sales['customer_id'].sample(frac=0.001, random_state=43).index
product_id_NA_idx = sales['product_id'].sample(frac=0.002, random_state=44).index
product_name_NA_idx = sales['product_name'].sample(frac=0.003, random_state=45).index

In [238]:
# criando valores vazios
sales['sale_id'].loc[sales_id_NA_idx] = np.nan
sales['customer_id'].loc[customer_id_NA_idx] = np.nan
sales['product_id'].loc[product_id_NA_idx] = np.nan
sales['product_name'].loc[product_name_NA_idx] = np.nan

C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\126450001.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['sale_id'].loc[sales_id_NA_idx] = np.nan
C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\126450001.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['customer_id'].loc[customer_id_NA_idx] = np.nan
C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\126450001.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sa

In [239]:
# verificando se há valores vazios novamente
sales.isna().sum()

sale_id          70
date              0
customer_id      64
product_id      121
quantity          0
price             0
product_name    186
dtype: int64

### **Gerando Outliers:**

In [240]:
# escolhendo os índices
qty_outlier9_idx = sales['quantity'].sample(frac=0.005, random_state=42).index
qty_outlier6_idx = sales['quantity'].sample(frac=0.002, random_state=43).index
qty_outlier30_idx = sales['quantity'].sample(frac=0.001, random_state=44).index
qty_outlier50_idx = sales['quantity'].sample(frac=0.001, random_state=45).index

In [241]:
# mudando os valores
sales['quantity'].loc[qty_outlier9_idx] = 9999
sales['quantity'].loc[qty_outlier6_idx] = 6666
sales['quantity'].loc[qty_outlier30_idx] = 30
sales['quantity'].loc[qty_outlier50_idx] = 50

C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\344872991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['quantity'].loc[qty_outlier9_idx] = 9999
C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\344872991.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['quantity'].loc[qty_outlier6_idx] = 6666
C:\Users\rapos\AppData\Local\Temp\ipykernel_10008\344872991.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['qu

In [ ]:
sales.sort_values(by='quantity', ascending=False)

,sale_id,date,customer_id,product_id,quantity,price,product_name
42085,950685-R7S8C,2020-12-22,325419-X5O31Y,71-Prs,9999,2 gp,"Pick, miner's"
31430,024030-5O46Q,2022-11-28,325826-5KZ3NW,101-Dnt,9999,6 gp,Drum
1566,304166-QAOP3,2022-06-11,325748-LG1ZHQ,03-Gns,9999,2 sp,Greatclub
23534,NaN,2022-02-22,325565-585I87,01-Por,9999,5 gp,Padded
15854,150554-HYTFL,2021-05-03,325432-EYCE4Q,39-Sus,9999,1 gp,Sprig of Mistletoe
...,...,...,...,...,...,...,...
30726,028226-YO6WC,2018-02-10,324825-5G10UR,184-ICo,1,"1,200 gp",Ioun Stone Protection
30746,028246-3ZYQ5,2018-05-03,324918-VN8LFJ,247-PCo,1,900 gp,Periapt of Wound Closure
30757,028257-78964,2019-10-20,325104-6WW8YL,06-Pon,1,100 gp,Potion of Poison
30768,028268-4YNKM,2017-10-14,324677-JXRPJ1,012-GCo,1,350 gp,Gem of Brightness


Agora o dataset está pronto para ser tramfsormado em um bando de dados SQL.

<hr style="height:3px;border-width:0;color:blue;background-color:blue">

# **Transformando em um BD SQLite.**

Como eu pretendo disponibilizar esse BD no Kaggle, eu vou tranformar em um BD SQLite por ser mais comum na plataforma.

**Minhas tabelas:**
* adventure_gear
* magic_items_filtered
* weapons
* potions_filtered
* poisons
* armor
* product
* sales
* costumers

In [250]:
import sqlite3

In [251]:
adventure_gear.head()

,item_id,name,price,weight,category,type
0,01-Ars,Abacus,2 gp,2 lb.,Others,adventure_gear
1,02-Ars,Acid (vial),25 gp,1 lb.,Others,adventure_gear
2,03-Ars,Alchemist's Fire (flask),50 gp,1 lb.,Others,adventure_gear
3,04-Aon,Arrows (20),1 gp,1 lb.,Ammunition,adventure_gear
4,05-Bon,Blowgun Needle (50),1 gp,1 lb.,Ammunition,adventure_gear


In [252]:
magic_items_filtered.head()

,item_id,name,price,rarity,category,type
0,001-ACo,Ammunition +1 (Per),15 gp,Uncommon,Consumable Items,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,Rare,Consumable Items,magic_item
4,005-BCo,Bead of Force,"1,000 gp",Rare,Consumable Items,magic_item
5,006-CCo,Chime of Opening,400 gp,Rare,Consumable Items,magic_item
6,007-DCo,Deck of Illusions,900 gp,Uncommon,Consumable Items,magic_item


In [253]:
weapons.head()

,item_id,name,price,damage,weight,properties,category,type
0,01-Cns,Club,1 sp,1d4 Bludgeon,2 lb.,Light,Simple Melee Weapons,weapon
1,02-Dns,Dagger,2 gp,1d4 Piercing,1 lb.,"Finesse, Light, Thrown (20/60)",Simple Melee Weapons,weapon
2,03-Gns,Greatclub,2 sp,1d8 Bludgeon,10 lb.,Two-handed,Simple Melee Weapons,weapon
3,04-Hns,Handaxe,5 gp,1d6 Slashing,2 lb.,"Light, Thrown (20/60)",Simple Melee Weapons,weapon
4,05-Jns,Javelin,5 sp,1d6 Piercing,2 lb.,Thrown (30/120),Simple Melee Weapons,weapon


In [254]:
armor.head()

,item_id,name,price,ac,weight,requirements,stealth,category,type
0,01-Por,Padded,5 gp,11 + Dex,8 lb.,NaN,Disadvantage,Light Armor,armor
1,02-Lor,Leather,10 gp,11 + Dex,10 lb.,NaN,NaN,Light Armor,armor
2,03-Sor,Studded Leather,45 gp,12 + Dex,13 lb.,NaN,NaN,Light Armor,armor
3,04-Hor,Hide,10 gp,12 + Dex(max2),12 lb.,NaN,NaN,Medium Armor,armor
4,05-Cor,Chain Shirt,50 gp,13 + Dex(max2),20 lb.,NaN,NaN,Medium Armor,armor


In [255]:
potions_filtered.head()

,item_id,name,price,rarity,type
0,01-Pon,Potion of Healing,50 gp,Common,potion
1,02-Pon,Potion of Greater Healing,150 gp,Uncommon,potion
2,03-Pre,Potion of Superior Healing,450 gp,Rare,potion
4,05-Ere,Elixir of Health,120 gp,Rare,potion
5,06-Pon,Potion of Poison,100 gp,Uncommon,potion


In [256]:
poisons.head()

,item_id,name,price,category,dc,type
0,01-Aed,Assassin's blood,150 gp,Ingested,10.0,poison
1,02-Ted,Truth serum,150 gp,Ingested,11.0,poison
2,03-Cct,Carrion crawler mucus,200 gp,Contact,13.0,poison
3,04-Dry,Drow poison,200 gp,Injury,13.0,poison
4,05-Sry,Serpent venom,200 gp,Injury,11.0,poison


In [257]:
product.head()

,product_id,product_name,price,type
0,001-ACo,Ammunition +1 (Per),15 gp,magic_item
1,002-ACo,Ammunition +2 (Per),50 gp,magic_item
2,005-BCo,Bead of Force,"1,000 gp",magic_item
3,006-CCo,Chime of Opening,400 gp,magic_item
4,007-DCo,Deck of Illusions,900 gp,magic_item


In [258]:
sales.head()

,sale_id,date,customer_id,product_id,quantity,price,product_name
0,019600-4WBAB,2019-01-03,328842-Z0I083,062-BNo,1,"2,000 gp",Broom of Flying
1,019601-Q0FOT,2020-10-10,328144-E58HUX,09-Sns,2,1 gp,Sickle
2,019602-10GDM,2018-03-21,328712-ZZKW3G,86-Srs,2,5 sp,Sealing wax
3,019603-IPC8Q,2018-11-15,327646-NKOMV2,19-Brs,4,5 sp,Blanket
4,019604-O6P43,2023-10-19,327979-DEJ31G,296-ICo,1,"2,500 gp",Instrument of the Bards - Canaith Mandolin


In [259]:
customers.head()

,customer_id,name,sex,race,age,class
0,327455-753LC5,Veklani Daargen,female,Elf,661,Warlock
1,327456-8DRC11,Kasaki Wygarthe,female,Half-Elf,100,Wizard
2,327457-ERTJ5P,Rosalyn Faringray,female,Halfling,40,Barbarian
3,327458-HT0HL9,Atalya Webb,female,Tiefling,55,Monk
4,327459-XPSEIM,Grenenzel Lyfalia,female,Half-Orc,43,Bard


In [260]:
# criando conexão
conn = sqlite3.connect("adventurer_mart.db")

Inserindo as tabelas no BD:

In [261]:
adventure_gear.to_sql('details_adventure_gear', conn, index=False, if_exists='replace')
magic_items_filtered.to_sql('details_magic_items', conn, index=False, if_exists='replace')
weapons.to_sql('details_weapons', conn, index=False, if_exists='replace')
armor.to_sql('details_armor', conn, index=False, if_exists='replace')
potions_filtered.to_sql('details_potions', conn, index=False, if_exists='replace')
poisons.to_sql('details_poisons', conn, index=False, if_exists='replace')
product.to_sql('all_products', conn, index=False, if_exists='replace')
sales.to_sql('sales', conn, index=False, if_exists='replace')
customers.to_sql('costumers', conn, index=False, if_exists='replace')


1423

In [262]:
#fechando a conexão
conn.close()

**FIM.**